<a href="https://colab.research.google.com/github/YaninaK/sales-forecast/blob/main/notebooks/02_Feature_extraction_tsfresh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sales forecast for 20 stores for for 31 days' period. 
## Feature extraction tsfresh

[EDA - omissions and cluster analysis](
https://colab.research.google.com/drive/1udCgyEt7lusSY43lnduVDEnZvdvVEt9h?usp=sharing)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone -q https://github.com/YaninaK/sales-forecast.git
!pip install -r sales-forecast/requirements_Colab.txt -q 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.2/358.2 KB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 KB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.2/136.2 KB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 103.0 MB/s eta 0:00:00


In [3]:
%cd sales-forecast

/content/sales-forecast


In [4]:
import sys
import os

sys.path.append(os.getcwd())
sys.path.append(os.path.join(os.getcwd(), "src", "sales_forecast"))

In [5]:
import random
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler

from data.johnson_su_transformation import JohnsonSU
from data.validation import train_validation_split
from data.clean_data import clean_outliers
from features.features_tsfresh import get_tsfresh_features
from features.build_dataset import get_features

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
PATH = '/content/drive/MyDrive/ML_projects/01_Time_series/sales_forecast/data/'
DEFAULT_RANDOM_SEED = 3

In [8]:
def set_all_seeds(seed=DEFAULT_RANDOM_SEED):
  random.seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)
  np.random.seed(seed)

set_all_seeds()

## 1. Data ingestion

In [9]:
data = pd.read_parquet(PATH + '02_intermediate/preprocessed_data.parquet.gzip')
data.columns = range(len(data.columns))

print(f'data.shape = {data.shape}\n')
data.head(2)

data.shape = (468, 20)



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
dt,,,,,,,,,,,,,,,,,,,,
2016-01-02,747.8,421.8,546.2,355.7,341.2,341.8,510.9,620.0,1603.4,726.9,656.7,477.2,656.5,697.0,518.4,728.1,343.8,328.5,617.2,370.3
2016-01-03,681.2,427.1,549.3,418.1,326.0,367.5,548.3,605.3,1608.6,677.0,674.4,446.7,685.1,711.2,528.4,758.0,361.6,331.1,523.4,382.3


In [10]:
clusters = pd.read_parquet(PATH + '02_intermediate/clusters.parquet.gzip')

print(f'clusters.shape = {clusters.shape}\n')
clusters.head(2)

clusters.shape = (20, 1)



,cluster
0,1
1,1


In [11]:
n_shops = data.shape[1]

## 2. Train test split

* 1st half of 2016 and 1st half of 2017

In [12]:
train_df, valid_df, train_df_past, valid_df_past = train_validation_split(data)
train_df.shape, valid_df.shape, train_df_past.shape, valid_df_past.shape

((127, 20), (27, 20), (154, 20), (27, 20))

## 3. Data transformations

### 3.1 Johnson SU transformation

In [13]:
scaler = JohnsonSU()
scaler.fit(train_df)

train_df = scaler.transform(train_df)
valid_df = scaler.transform(valid_df)

X_scaled = pd.concat([train_df, valid_df], axis=0)

In [14]:
scaler_past = JohnsonSU()
scaler_past.fit(train_df_past)

X_past_scaled = scaler_past.transform(train_df_past)

### 3.2 Clean outliers

In [15]:
X_scaled = clean_outliers(X_scaled)

Number of outliers = 8


## 4. Extract features tsfresh

In [16]:
extracted_features = get_tsfresh_features(X_scaled)
extracted_features.shape

Feature Extraction: 100%|██████████| 20/20 [00:02<00:00,  7.50it/s]


(20, 643)

In [17]:
clusters = pd.get_dummies(clusters['cluster'])
extracted_features = pd.concat([clusters, extracted_features], axis=1)
extracted_features.shape

(20, 648)

In [18]:
sd_scaler = StandardScaler()
extracted_features = sd_scaler.fit_transform(extracted_features)

## 5. Build dataset

In [19]:
X = get_features(X_scaled, X_past_scaled, extracted_features)
X.shape

(154, 20, 650)

## 6. Save artifacts

In [20]:
np.save(PATH + '03_primary/dataset', X)
joblib.dump(scaler, PATH + '03_primary/scaler.joblib', compress=3)

['/content/drive/MyDrive/ML_projects/01_Time_series/sales_forecast/data/03_primary/scaler.joblib']

## 7. Load artifacts

In [21]:
X_new = np.load(PATH + '03_primary/dataset.npy')
X_new.shape

(154, 20, 650)

In [22]:
scaler_new = joblib.load(PATH + '03_primary/scaler.joblib')